# OPTIMIZING MCX GATES - PREPARING FOR FUTURE HARDWARE TODAY

This note describes how to use the Classiq platform to create MCX gates, including one with 14 controls. Then, it demonstrates a much more complex example with 50 control qubits.

## QUANTUM RESOURCES ARE VALUABLE, YET LIMITED

Quantum computers offer tantalizing promises to those who can harness their power. And though today’s computers are not quite able to solve real-world problems, those who are able to optimize for the hardware available will be able to reap rewards sooner than those who wait. The MCX gate is an important quantum gate used in a variety of circuits, such as the Grover Operator, logical AND operator, various state preparation algorithms, and arithmetic comparators. The ability to adapt implementations of MCX gates to meet the hardware constraints - limited qubit count, fidelities, gate count, and so on - is not trivial.

## HOW TO CREATE A 14-CONTROL MCX GATE WITH CLASSIQ

To create an MCX gate with 14 control qubits using Classiq, we simply call the MCX function and declare the control qubit count. Then, to constrain this circuit to only 20 qubits and optimize for circuit depth, we pass the max width and optimization parameter to the synthesis request.

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.builtin_functions import Mcx
from classiq.model import Constraints, OptimizationParameter

model = Model(
    constraints=Constraints(
        max_width=20, optimization_parameter=OptimizationParameter.DEPTH
    )
)
mcx_params = Mcx(num_ctrl_qubits=14)
model.Mcx(params=mcx_params)

qmod = model.get_model()
with open("mcx_14_ctrl_depth.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(qmod)
circuit = GeneratedCircuit.from_qprog(qprog)

print(f"Synthesized MCX depth is {circuit.transpiled_circuit.depth}")
circuit.show()

## AN MCX FOR EVERY OCCASION

The Classiq automatically optimizes the quantum circuit and each MCX gate to a plethora of possible situations. To characterize each setting we pass our constraints and preferences to the synthesis request.

### OPTIMIZING FOR DIFFERENT HARDWARE

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.builtin_functions import Mcx
from classiq.model import Constraints, OptimizationParameter, Preferences

model = Model(
    constraints=Constraints(optimization_parameter=OptimizationParameter.DEPTH),
    preferences=Preferences(
        backend_service_provider="IBM Quantum", backend_name="ibmq_kolkata"
    ),
)
mcx_params = Mcx(num_ctrl_qubits=14)
model.Mcx(params=mcx_params)

qmod = model.get_model()
with open("mcx_14_ctrl_hardware.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(qmod)
circuit = GeneratedCircuit.from_qprog(qprog)

print(f"Synthesized MCX depth is {circuit.transpiled_circuit.depth}")
circuit.show()

### OPTIMIZING FOR CX GATES

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.builtin_functions import Mcx
from classiq.model import Constraints, CustomHardwareSettings, Preferences

model = Model(
    constraints=Constraints(max_width=19, optimization_parameter="cx"),
    preferences=Preferences(
        custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"])
    ),
)
mcx_params = Mcx(num_ctrl_qubits=14)
model.Mcx(params=mcx_params)

qmod = model.get_model()
with open("mcx_14_ctrl_cx.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(qmod)
circuit = GeneratedCircuit.from_qprog(qprog)

print(f"Synthesized MCX cx-count is {circuit.transpiled_circuit.count_ops['cx']}")
circuit.show()

## BEYOND 14 CONTROLS

The power of the Classiq synthesis engine is far greater than creating optimized, 14-control MCX gates in an instant. Take the following code, where we create an MCX gate with 50 control qubits.

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.builtin_functions import Mcx

model = Model()
mcx_params = Mcx(num_ctrl_qubits=50)
model.Mcx(params=mcx_params)

qmod = model.get_model()
with open("mcx_50_ctrl.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(qmod)
circuit = GeneratedCircuit.from_qprog(qprog)

circuit.show()

![MCX depth](../../../resources/images/MCX_graph_1_dec_22.png)
